In [2]:
import numpy as np
from scipy.stats import norm

def black_scholes_swaption_pricer(swap_rate, strike_rate, volatility, maturity, notional, is_receiver=True):
    d1 = (np.log(swap_rate / strike_rate) + 0.5 * volatility**2 * maturity) / (volatility * np.sqrt(maturity))
    d2 = d1 - volatility * np.sqrt(maturity)

    if is_receiver:
        option_price = notional * (swap_rate * norm.cdf(d1) - strike_rate * norm.cdf(d2))
    else:
        option_price = notional * (strike_rate * norm.cdf(-d2) - swap_rate * norm.cdf(-d1))

    return option_price

# Example usage:
swap_rate = 0.02  # Current swap rate
strike_rate = 0.025  # Swaption strike rate
volatility = 0.2  # Volatility of the underlying interest rate
maturity = 2  # Time to expiration in years
notional = 100  # Notional amount of the swaption

# Receiver swaption
receiver_swaption_price = black_scholes_swaption_pricer(swap_rate, strike_rate, volatility, maturity, notional, is_receiver=True)
print(f"Receiver Swaption Price: {receiver_swaption_price:.2f}")

# Payer swaption
payer_swaption_price = black_scholes_swaption_pricer(swap_rate, strike_rate, volatility, maturity, notional, is_receiver=False)
print(f"Payer Swaption Price: {payer_swaption_price:.2f}")


Receiver Swaption Price: 0.08
Payer Swaption Price: 0.58


In [3]:
import pandas as pd
from math import log, sqrt, exp
from scipy.stats import norm

# Black-Scholes formula for European call option price
def black_scholes_call(S, K, r, T, sigma):
    d1 = (log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    call_price = S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
    return call_price

# Black-Scholes formula for European put option price
def black_scholes_put(S, K, r, T, sigma):
    call_price = black_scholes_call(S, K, r, T, sigma)
    put_price = call_price - S + K * exp(-r * T)
    return put_price

# SABR model for implied volatility
def sabr_volatility(F, K, T, alpha, beta, rho, nu):
    X = K
    if F != K:
        z = (F * X)**((1 - beta) / 2) * log(F / X)
        x = log((sqrt(1 - 2 * rho * z + z**2) + z - rho) / (1 - rho))
        factor1 = (F * X)**((1 - beta) / 2)
        factor2 = 1 + ((1 - beta)**2 * log(F / X)**2) / 24 + ((1 - beta)**4 * log(F / X)**4) / 1920
        sigma = alpha * factor1 * factor2 * (1 + ((1 - beta)**2 / 24 * alpha**2 / (factor1**2)) + (1 - beta)**4 / 1920 * (alpha**4 / factor1**4)) * nu / x
    else:
        factor1 = (F * X)**((1 - beta) / 2)
        factor2 = 1 + ((1 - beta)**2 * log(F / X)**2) / 24 + ((1 - beta)**4 * log(F / X)**4) / 1920
        sigma = alpha * factor1 * factor2
    return sigma

# Calculate the price of a swaption
def calculate_swaption_price(swap_rate, strike_price, expiration, tenor, alpha, beta, rho, nu, risk_free_rate):
    T = expiration + tenor
    implied_volatility = sabr_volatility(swap_rate, strike_price, T, alpha, beta, rho, nu)
    swaption_call_price = black_scholes_call(swap_rate, strike_price, risk_free_rate, T, implied_volatility)
    swaption_put_price = black_scholes_put(swap_rate, strike_price, risk_free_rate, T, implied_volatility)
    return swaption_call_price, swaption_put_price, implied_volatility

# Calculate the price of a straddle
def calculate_straddle_price(swap_rate, strike_price, expiration, tenor, alpha, beta, rho, nu, risk_free_rate):
    T = expiration + tenor
    implied_volatility = sabr_volatility(swap_rate, strike_price, T, alpha, beta, rho, nu)
    straddle_call_price = black_scholes_call(swap_rate, strike_price, risk_free_rate, T, implied_volatility)
    straddle_put_price = black_scholes_put(swap_rate, strike_price, risk_free_rate, T, implied_volatility)
    straddle_price = straddle_call_price + straddle_put_price
    return straddle_price, implied_volatility

# Calculate prices for multiple swaptions or straddles using pandas DataFrame
def calculate_prices(data):
    prices = pd.DataFrame(columns=['Swaption Call Price', 'Swaption Put Price', 'Implied Volatility', 'Straddle Price'])
    for index, row in data.iterrows():
        swap_rate = row['Swap Rate']
        strike_price = row['Strike Price']
        expiration = row['Expiration']
        tenor = row['Tenor']
        alpha = row['Alpha']
        beta = row['Beta']
        rho = row['Rho']
        nu = row['Nu']
        risk_free_rate = row['Risk-Free Rate']

        swaption_call_price, swaption_put_price, implied_volatility = calculate_swaption_price(
            swap_rate, strike_price, expiration, tenor, alpha, beta, rho, nu, risk_free_rate
        )
        straddle_price, implied_volatility_straddle = calculate_straddle_price(
            swap_rate, strike_price, expiration, tenor, alpha, beta, rho, nu, risk_free_rate
        )

        prices.loc[index] = [swaption_call_price, swaption_put_price, implied_volatility, straddle_price]

    return prices

# Example usage with a pandas DataFrame
data = pd.DataFrame({
    'Swap Rate': [100.0, 105.0, 98.5],
    'Strike Price': [98.0, 100.0, 105.0],
    'Expiration': [0.5, 0.75, 1.0],
    'Tenor': [3, 6, 9],
    'Alpha': [0.2, 0.25, 0.3],  # SABR parameter alpha
    'Beta': [0.5, 0.5, 0.5],  # SABR parameter beta
    'Rho': [0.1, -0.2, 0.3],  # SABR parameter rho
    'Nu': [0.4, 0.3, 0.2],  # SABR parameter nu
    'Risk-Free Rate': [0.05, 0.06, 0.04]
})

prices = calculate_prices(data)
print(prices)


   Swaption Call Price  Swaption Put Price  Implied Volatility  Straddle Price
0            99.979812           82.246600            3.946731      182.226412
1           102.434678           64.132359            1.662651      166.567038
2           -62.893234          -91.009629           -1.071368     -153.902863


In [4]:
data

,Swap Rate,Strike Price,Expiration,Tenor,Alpha,Beta,Rho,Nu,Risk-Free Rate
0,100.0,98.0,0.50,3,0.20,0.5,0.1,0.4,0.05
1,105.0,100.0,0.75,6,0.25,0.5,-0.2,0.3,0.06
2,98.5,105.0,1.00,9,0.30,0.5,0.3,0.2,0.04
